# Detector Comparison Summary

Compare all pluggable detector models on sequential fault simulation.

In [1]:
import os
import json
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings('ignore')

METRICS_DIR = Path('../outputs/metrics')
FIGURES_DIR = Path('../outputs/figures')

QUICK_MODE = os.environ.get('QUICK_MODE', '').lower() in ('true', '1', 'yes')
FILE_SUFFIX = '_quick' if QUICK_MODE else ''

print('='*60)
print('Detector Comparison Summary')
if QUICK_MODE:
    print('QUICK MODE')
print('='*60)

Detector Comparison Summary


In [2]:
print('\nLoading metrics...')

models = ['xgboost', 'lstm', 'lstm_fcn', 'cnn_transformer', 'transkal', 'lstm_autoencoder', 'conv_autoencoder']
metrics = {}

for model in models:
    file = METRICS_DIR / f'{model}_detector_results{FILE_SUFFIX}.json'
    if file.exists():
        with open(file) as f:
            metrics[model] = json.load(f)
        print(f'  Loaded {model}')
    else:
        print(f'  Missing {model}')

print(f'\nLoaded {len(metrics)} models')


Loading metrics...
  Loaded xgboost
  Loaded lstm
  Loaded lstm_fcn
  Loaded cnn_transformer
  Loaded transkal
  Loaded lstm_autoencoder
  Loaded conv_autoencoder

Loaded 7 models


In [3]:
print('\nModel Comparison:')
print('='*80)

rows = []
for model, m in metrics.items():
    if 'overall_metrics' in m:  # Multiclass classifier format
        rows.append({
            'Model': m['model'],
            'Type': 'Classifier',
            'Accuracy': m['overall_metrics']['accuracy'],
            'F1 (weighted)': m['overall_metrics'].get('f1_weighted', 0),
            'Fault Detection Recall': m['overall_metrics'].get('fault_detection_recall', 0)
        })
    elif 'metrics' in m:  # Binary autoencoder format
        rows.append({
            'Model': m['model'],
            'Type': 'Anomaly Detector',
            'Accuracy': m['metrics']['accuracy'],
            'F1 (weighted)': m['metrics'].get('f1', 0),
            'Fault Detection Recall': m['metrics'].get('recall', 0)
        })

df = pd.DataFrame(rows)
print(df.to_string(index=False))

df.to_csv(METRICS_DIR / f'detector_comparison{FILE_SUFFIX}.csv', index=False)


Model Comparison:
           Model             Type  Accuracy  F1 (weighted)  Fault Detection Recall
         XGBoost       Classifier  0.446857       0.516891                0.848739
            LSTM       Classifier  0.331429       0.375257                0.781513
        LSTM-FCN       Classifier  0.321143       0.370931                0.837535
 CNN-Transformer       Classifier  0.416000       0.436806                0.761905
        TransKal       Classifier  0.132571       0.128588                1.000000
LSTM-Autoencoder Anomaly Detector  0.472000       0.606474                0.997199
Conv-Autoencoder Anomaly Detector  0.452571       0.598491                1.000000


In [4]:
print('\n' + '='*60)
print('Comparison Complete!')
print('='*60)


Comparison Complete!
